In [1]:
import cv2
import numpy as np
from PIL import Image

### We are going to work with HSV colorspace for color detection, because HSV simplifies and improves color detection, making it more accurate and adaptable to lighting variations. That’s why OpenCV and most computer vision tasks prefer HSV over RGB. HSV stands for:  

- **H (Hue)** → The actual color (Red, Blue, Green, etc.). Measured in degrees **(0° - 360°)**.  
- **S (Saturation)** → The intensity or purity of the color (**0% - 100%** or **0 - 255** in OpenCV).  
- **V (Value or Brightness)** → The brightness of the color (**0% - 100%** or **0 - 255** in OpenCV).  
 
In the **Hue, Saturation, and Value (HSV)** color space, you need both **upper and lower limits** to define a specific color range. Each color occupies a range of **H, S, and V**, so to detect a specific color, you define:  

- **Lower Bound** → Minimum values of **H, S, and V**  
- **Upper Bound** → Maximum values of **H, S, and V**  

This makes HSV an efficient and accurate choice for **color-based segmentation and detection** in computer vision. 

In [2]:
# Function that takes BGR color and returns the color's Lower and upper limits:

def get_limit(bgr_color, hue_range=10, sat_range=50, val_range=50):

    # Convert BGR to HSV using OpenCV
    hsv_color = cv2.cvtColor(np.uint8([[bgr_color]]), cv2.COLOR_BGR2HSV)[0][0]
    
    # Extract H, S, V values
    h, s, v = hsv_color

    # Define lower and upper limits with a range
    lower_hsv = np.array([
        max(h - hue_range, 0),      # Lower Hue
        max(s - sat_range, 0),      # Lower Saturation
        max(v - val_range, 0)])     # Lower Value

    upper_hsv = np.array([
        min(h + hue_range, 179),    # Upper Hue (max 179 in OpenCV)
        min(s + sat_range, 255),    # Upper Saturation (max 255)
        min(v + val_range, 255)])   # Upper Value (max 255)

    return lower_hsv, upper_hsv

**Understanding HSV Range for Color Detection**  

When detecting a color, we define an **acceptable range** around its **Hue (H), Saturation (S), and Value (V)** to improve accuracy.  

**1️⃣ Hue (H)**  
- `h - hue_range` → Allows similar hues **before** the actual color.  
- `h + hue_range` → Allows similar hues **after** the actual color.  
- `max(..., 0)` → Prevents negative values (**Hue range: 0–179 in OpenCV**).  
- `min(..., 179)` → Ensures Hue does not exceed **179**.  

**2️⃣ Saturation (S) and Value (V)**  
- A small range (`sat_range`, `val_range`) is added/subtracted to detect colors even if they are **slightly desaturated or darker**.  
- `max(..., 0)` → Prevents negative values.  
- `min(..., 255)` → Ensures values do not exceed **255**.  

By defining these upper and lower HSV limits, we can **reliably detect a color** even under varying lighting conditions.

In [3]:
given_color = [192, 00, 0]    # blue in BGR colorspace

In [4]:
vid = cv2.VideoCapture(0)

while True:
    ret, frame = vid.read()         # reads and returns frame, and gives ret as True if frame is read correctly or False if not read.

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)     # Converts the frame from BGR to HSV image.

    lower_limit, upper_limit = get_limit(bgr_color = given_color)      # Gets the limits from the function defined above.

    mask = cv2.inRange(hsv_frame, lower_limit, upper_limit)      # .inRange() is used to create a binary mask based on a given color range.

    mask_ = Image.fromarray(mask)      #  .fromarray() function in PIL is used to convert a NumPy array(OpenCV binary mask) into a Pillow Image object.

    box = mask_.getbbox()       # .getbbox() method in PIL (Pillow) is used to find the bounding box of non-zero (non-black) pixels in an image.

    if box is not None:
        x1, y1, x2, y2 = box    # Unpacks the bounding box coordinates (top-left and bottom-right corners).

        frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)   # Draws a green rectangle around the detected object.
    
    cv2.imshow('Color Detection Webcam', frame)      # Displays the processed video frame in a window named "Color Detection Webcam".
    if cv2.waitKey(1) & 0xFF == ord('q'):           # Waits for a key press for 1 millisecond and checks if the 'q' key was pressed. 
        break                                       # If 'q' is pressed, loop terminates/break.

vid.release()
cv2.destroyAllWindows()

**Syntax:** .inRange(src, lowerb, upperb), Parameters:

**src:** The source image (typically in HSV or grayscale format).
**lowerb:** The lower bound of the color range.
**upperb:** The upper bound of the color range.

Every pixel in src that falls within the range [lowerb, upperb] is set to 255 (white) in mask.
All other pixels are set to 0 (black).

**Syntax:** box = image.getbbox()

**How It Works?**
It scans the image and finds the smallest bounding box (rectangle) that contains all the non-black (non-zero) pixels.
If the image is completely black, it returns None.